In [1]:
from collections import deque
from queue import PriorityQueue

def calculate_distance(point1, point2, algorithm):
    x1, x2, y1, y2, z1, z2=  float(point1[0]), float(point2[0]), float(point1[1]), float(point2[1]), float(point1[2]), float(point2[2])
    if algorithm == 'A*':
        return ((x2 - x1)**2 + (y2 - y1)**2 + (z2- z1)**2)**0.5
    elif algorithm=='UCS':
        return ((x2 - x1)**2 + (y2 - y1)**2)**0.5

    return 1

def heuristic(current_node, goal):
    return calculate_distance(current_node, goal, 'A*')

def read_input_file(file_path):

    file =  open(file_path, "r")
    input_data = file.readlines()

    algorithm = input_data[0].strip('\n')
    energy_limit = int(input_data[1].strip('\n'))

    num_safe_locations = int(input_data[2][:-1])
    safe_locations = [i.strip('\n') for i in input_data[3:num_safe_locations+3]]
    safe_locations_dict = {}
    for i in safe_locations:
        safe_locations_dict[i.split(' ')[0]] = i.split(' ')[1:]

    num_safe_path_segments = int(input_data[num_safe_locations+3][:-1])
    safe_path_segments = [j.strip('\n') for j in input_data[num_safe_locations+4:]]
    graph = {}
    count=0
    for segment in safe_path_segments:
        node, neighbor = segment.split(' ')
        try:
            graph[node][neighbor] = calculate_distance(safe_locations_dict[node], safe_locations_dict[neighbor], algorithm)
        except:
            graph[node] = {}
            graph[node][neighbor] = calculate_distance(safe_locations_dict[node], safe_locations_dict[neighbor], algorithm)

        try:
            graph[neighbor][node] = calculate_distance(safe_locations_dict[node], safe_locations_dict[neighbor], algorithm)
        except:
            graph[neighbor] = {}
            graph[neighbor][node] = calculate_distance(safe_locations_dict[node], safe_locations_dict[neighbor], algorithm)

    return algorithm, energy_limit, safe_locations_dict, graph

def read_output_file(file_path):

  file =  open(file_path, "r")
  output_data = file.readlines()
  true_ans = []
  for line in output_data:
    for node in line.strip('\n').split(' '):
      true_ans.append(node)

  return true_ans[:-1]

def bfs_final(graph, start, goal, safe_locations_dict, motor_energy):
    queue = deque([(start, ['start'], 0)])
    visited = {}
    while queue:
        current_node, path, momentum = queue.popleft()
        if current_node==goal:
            return path
        if current_node in visited and visited[current_node]>=momentum:
            continue
        visited[current_node] = momentum
        for neighbor in graph[current_node]:
            z_node = safe_locations_dict[current_node][-1]
            z_neighbor = safe_locations_dict[neighbor][-1]
            energy_reqd = int(z_neighbor) - int(z_node)
            new_momentum = max(0,-energy_reqd)
            if neighbor not in visited or visited[neighbor]<new_momentum:
                if energy_reqd<=(motor_energy + momentum):
                    new_path = path + [neighbor]
                    queue.append((neighbor, new_path, new_momentum))
    return ['FAIL']

def ucs_final(graph, start, goal, safe_locations_dict, motor_energy):
    queue = PriorityQueue()
    queue.put((0, start, [start], 0))
    visited = {}
    while not queue.empty():
        cost, current_node, path, momentum = queue.get()
        if current_node==goal:
            return path, cost
        if current_node in visited and visited[current_node]>=momentum:
            continue
        visited[current_node] = momentum
        for neighbor, neighbor_cost in graph[current_node].items():
            z_node = safe_locations_dict[current_node][-1]
            z_neighbor = safe_locations_dict[neighbor][-1]
            energy_reqd = int(z_neighbor) - int(z_node)
            new_momentum = max(0,-energy_reqd)
            if neighbor not in visited or visited[neighbor]<new_momentum:
                if energy_reqd<=(motor_energy + momentum):
                    total_cost = cost + neighbor_cost
                    queue.put((total_cost, neighbor, path + [neighbor], new_momentum))
    return ['FAIL']

def a_star_final(graph, start, goal, safe_locations_dict, motor_energy):
    queue = PriorityQueue()
    queue.put((0, 0, start, [start], 0))
    visited = {}
    while not queue.empty():
        _, cost, current_node, path, momentum = queue.get()
        if current_node==goal:
            return path, cost
        if current_node in visited and visited[current_node]>=momentum:
            continue
        visited[current_node] = momentum
        for neighbor, neighbor_cost in graph[current_node].items():
            z_node = safe_locations_dict[current_node][-1]
            z_neighbor = safe_locations_dict[neighbor][-1]
            energy_reqd = int(z_neighbor) - int(z_node)
            new_momentum = max(0,-energy_reqd)
            if neighbor not in visited or visited[neighbor]<new_momentum:
                if energy_reqd<=(motor_energy + momentum):
                    cost_till_neighbor = cost + neighbor_cost
                    total_cost = cost_till_neighbor + heuristic(safe_locations_dict[neighbor], safe_locations_dict[goal])
                    queue.put((total_cost, cost_till_neighbor, neighbor, path + [neighbor], new_momentum))
    return ['FAIL']

In [2]:
import os
folder_path = '/content/drive/MyDrive/USC_CS_AI/CS561_AI/HW/HW1_Search/codes/testcases/'


for i in range(1,51):
    file_path = folder_path + 'input' + str(i) + '.txt'
    algorithm, motor_energy, safe_locations_dict, graph = read_input_file(file_path)

    print(i, 'File:', file_path.split('/')[-1])

    if algorithm=='BFS':
        print('BFS')
        ans = bfs_final(graph, 'start', 'goal', safe_locations_dict, motor_energy)
        print('My Answer:')
        print(len(ans), ans)

        output_file = folder_path + 'output' + str(i) + '.txt'
        true_ans = read_output_file(output_file)
        print('True Ans:')
        print(len(true_ans), true_ans)
        print()

    if algorithm=='UCS':
        print("UCS")
        ans = ucs_final(graph, 'start', 'goal', safe_locations_dict, motor_energy)
        print('My Answer:')
        print(len(ans[0]), ans)

        output_file = folder_path + 'output' + str(i) + '.txt'
        true_ans = read_output_file(output_file)
        print('True Ans:')
        print(len(true_ans), true_ans)
        output_length_file = folder_path + 'pathlen' + str(i) + '.txt'
        file =  open(output_length_file, "r")
        input_data = file.readlines()

        print('My paths cost:', round(ans[1], 2))
        print('True Paths cost:', input_data[0].strip('\n'))
        print()

    if algorithm=='A*':
        print('A*')
        ans = a_star_final(graph, 'start', 'goal', safe_locations_dict, motor_energy)
        print('My Answer:')
        print(len(ans[0]), ans)

        output_file = folder_path + 'output' + str(i) + '.txt'
        true_ans = read_output_file(output_file)
        print('True Ans:')
        print(len(true_ans), true_ans)
        output_length_file = folder_path + 'pathlen' + str(i) + '.txt'
        file =  open(output_length_file, "r")
        input_data = file.readlines()

        print('My paths cost:', round(ans[1], 2))
        print('True Paths cost:', input_data[0].strip('\n'))
        print()

    print()

1 File: input1.txt
BFS
My Answer:
23 ['start', 'qdwa', 'wes', 'xwv', 'qk', 'pp', 'bcwz', 'kgi', 'bcwz', 'jknjn', 'vspp', 'sdru', 'qklo', 'zb', 'onbw', 'fnuj', 'xcf', 'hbzcw', 'fymx', 'dus', 'ube', 'ryo', 'goal']
True Ans:
23 ['start', 'qdwa', 'wes', 'gpco', 'fmzr', 'eniu', 'hwghh', 'vdn', 'vry', 'dcd', 'efbk', 'jdcy', 'jnqzm', 'fxra', 'ivy', 'fxra', 'pgj', 'dytx', 'cwv', 'khz', 'ftv', 'hxouz', 'goal']


2 File: input2.txt
UCS
My Answer:
31 (['start', 'zl', 'qypez', 'zl', 'lnrf', 'zl', 'fenoa', 'sbcp', 'qed', 'ncbgo', 'ktmgl', 'ug', 'wns', 'nfw', 'eada', 'ta', 'djst', 'nyyvh', 'yscls', 'fm', 'uxe', 'mw', 'dtba', 'kgy', 'hza', 'cvuuk', 'uq', 'cvuuk', 'pikcv', 'myujd', 'goal'], 5877.934425231435)
True Ans:
31 ['start', 'zl', 'qypez', 'zl', 'lnrf', 'zl', 'fenoa', 'sbcp', 'qed', 'ncbgo', 'ktmgl', 'ug', 'wns', 'nfw', 'eada', 'ta', 'djst', 'nyyvh', 'yscls', 'fm', 'uxe', 'mw', 'dtba', 'kgy', 'hza', 'cvuuk', 'uq', 'cvuuk', 'pikcv', 'myujd', 'goal']
My paths cost: 5877.93
True Paths cost: 5877.9